This notebook does standard adversarial training as a baseline.

# Setup

### Import dependencies

In [1]:
import os
os.environ["NOTEBOOK_MODE"] = "1" # for robustness library

from robustness import model_utils, datasets, train, defaults
from robustness.datasets import CIFAR

# We use cox (http://github.com/MadryLab/cox) to log, store and analyze
# results. Read more at https//cox.readthedocs.io.
from cox.utils import Parameters
import cox.store

import torch
torch.cuda.get_device_name(0)

/ufs/ttw/quada/miniconda3/envs/robustness/lib/python3.6/site-packages/robustness/train.py:24: UserWarning: Could not import amp.
  warnings.warn('Could not import amp.')


'TITAN RTX'

### Load dataset and create model

In [2]:
DS = CIFAR("data")
TRAIN_LOADER, VAL_LOADER = DS.make_loaders(
    batch_size=128, workers=12
)

==> Preparing dataset cifar..
Files already downloaded and verified
Files already downloaded and verified


In [3]:
m_resnet50, _ = model_utils.make_and_restore_model(
    arch="resnet50",
    dataset=DS, 
)

### Set up Cox logging

In [4]:
COX_STORE = cox.store.Store("cox")

Logging in: /fs/data/ttw/code/adversarial-ntks/notebooks/discard-hypothesis/cox/cef3bae3-1dfc-4b8b-a787-0ca7f424f8ea


# Accuracy at init

In [5]:
EVAL_ARGS = defaults.check_and_fill_args(
    args=Parameters({
        "adv_eval": 1,
        "out_dir": "eval_out",
        "constraint": 'inf', # L-inf PGD
        "eps": 8.0 / 255.0, # Epsilon constraint (L-inf norm)
        "attack_lr": 2.5 * 8 / 255 / 20,
        "attack_steps": 20
    }),
    arg_list=defaults.PGD_ARGS,
    ds_class=CIFAR
)

In [6]:
train.eval_model(
    model=m_resnet50,
    args=EVAL_ARGS,
    loader=VAL_LOADER,
    store=COX_STORE
)

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

{'epoch': 0,
 'nat_prec1': tensor(9.5500, device='cuda:0'),
 'adv_prec1': tensor(1.2600, device='cuda:0'),
 'nat_loss': 2.3029249923706057,
 'adv_loss': 2.308146116256714,
 'train_prec1': nan,
 'train_loss': nan,
 'time': 154.21001958847046}

# Adversarial finetuning 

In [7]:
TRAIN_ARGS = Parameters({
    "out_dir": "train_out",
    "adv_train": 1, # Use adversarial training
    "constraint": 'inf', # L-inf PGD
    "eps": 8.0 / 255.0, # Epsilon constraint (L-inf norm)
    "attack_lr": 2.5 * 8 / 255 / 20,
    "attack_steps": 20
})

# Fill whatever parameters are missing from the defaults
TRAIN_ARGS = defaults.check_and_fill_args(
    TRAIN_ARGS, defaults.TRAINING_ARGS, CIFAR
)
TRAIN_ARGS = defaults.check_and_fill_args(
    TRAIN_ARGS, defaults.PGD_ARGS, CIFAR
)

In [ ]:
# Train a model
train.train_model(
    model=m_resnet50,
    update_params=m_resnet50.model.parameters(),
    args=TRAIN_ARGS,
    loaders=(TRAIN_LOADER, VAL_LOADER),
    store=COX_STORE
)

  0%|          | 0/391 [00:00<?, ?it/s]